In [11]:
import httpx
from bs4 import BeautifulSoup
import asyncio

In [12]:
async def get_response(client: httpx.AsyncClient, url: str) -> httpx.Response:
    response = await client.get(url)
    return response

In [13]:
urls = [
    "https://www.example.com",
    "https://www.oreilly.com",
    "https://www.python.org",
    "https://www.github.com",
    "https://www.stackoverflow.com",
    "https://www.reddit.com",
    "https://www.medium.com",
    "https://www.wikipedia.org",
    "https://www.linkedin.com",
    "https://www.twitter.com"
]

async with httpx.AsyncClient() as client:
    tasks = [asyncio.create_task(get_response(client, url)) for url in urls]
    responses = await asyncio.gather(*tasks)

In [16]:
with httpx.Client() as client:
    responses = [client.get(url) for url in urls]

In [21]:
responses[1].url

URL('https://www.oreilly.com')